In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image

device = "cuda:0"

modelPath = "nikravan/glm-4vq"
tokenizer = AutoTokenizer.from_pretrained(modelPath, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    modelPath,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    device_map="auto",
    use_cache=True,
)

/home/boom/.pyenv/versions/aivet/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/boom/.pyenv/versions/aivet/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

/home/boom/.pyenv/versions/aivet/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
query = """Task: Extract and list all blood test details.

Objective:
1. Identify and extract **chemical names** from the blood test report.
2. Capture the **detected levels** of each chemical.
3. Extract the **normal range** for each corresponding chemical.

Expected Output Format:
- Chemical Name: [Name]
  - Detected Level: [Value]
  - Normal Range: [Min Value] - [Max Value]

Please ensure accurate extraction, including any unit symbols (e.g., mg/dL, IU/L), and handle variations in formatting or alignment. Return results in a structured list format for easy readability."""
image = Image.open("data/ref1.png").convert("RGB")
# image=""
inputs = tokenizer.apply_chat_template(
    [{"role": "user", "image": image, "content": query}],
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt",
    return_dict=True,
)  # chat with image mode

inputs = inputs.to(device)

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1, "temperature": 0.2}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs["input_ids"].shape[1] :]
    print(tokenizer.decode(outputs[0]))

Certainly, here is a structured list of blood test details from the provided report:

- Chemical Name: Creatine Kinase (CPK) (IU/L)
  - Detected Level: 217
  - Normal Range: 45 - 155
- Chemical Name: Alanine Aminotransferase (AST) (IU/L)
  - Detected Level: 36
  - Normal Range: 10 - 45
- Chemical Name: Aspartate Aminotransferase (ALT) (IU/L)
  - Detected Level: 83
  - Normal Range: 10 - 60
- Chemical Name: Alkaline Phosphatase (ALP) (IU/L)
  - Detected Level: 2780
  - Normal Range: 45 - 152
- Chemical Name: Gamma-Glutamyl Transpeptidase (GGT) (IU/L)
  - Detected Level: 3,0
  - Normal Range: 0,1 - 13
- Chemical Name: Bilirubina Total (mg/dl)
  - Detected Level: 0,10
  - Normal Range: 0,10 - 0,44
- Chemical Name: Protein Total (g/dl)
  - Detected Level: 7,2
  - Normal Range: 5,8 - 8,0
- Chemical Name: Albumine (g/dl)
  - Detected Level: 3,26
  - Normal Range: 2,6 - 3,8
- Chemical Name: Globuline (g/dl)
  - Detected Level: 3,90
  - Normal Range: 2,6 - 4,5
- Chemical Name: Rapp. Alb/Glob
 

In [ ]:
query = "Extract all the blood test chemical name values, their detected level and the corresponding normal range"
image = Image.open("data/ref1.png").convert("RGB")
inputs = tokenizer.apply_chat_template(
    [{"role": "user", "image": image, "content": query}],
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt",
    return_dict=True,
)  # chat with image mode

inputs = inputs.to(device)

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1, "temperature": 0.2}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs["input_ids"].shape[1] :]
    print(tokenizer.decode(outputs[0]))

Based on the blood test results provided, the following molecules have detected values that are outside the normal range:

1. **Creatinine (CREA, mg/dl):** The detected value is 21, which is outside the normal range of 0.69 - 1.80.
2. **Urea (UREA, mg/dl):** The detected value is 350, which is outside the normal range of 15 - 45.
3. **Glucose (GLUCOSIO, mg/dl):** The detected value is 88, which is outside the normal range of 70 - 110.
4. **Potassium (POTASSIO, mEq/L):** The detected value is 4.9, which is outside the normal range of 3.3 - 5.4.
5. **Magnesium (MAGNESIO, mg/dl):** The detected value is 1.50, which is outside the normal range of 1.34 - 2.48.
6. **Alanine aminotransferase (ALT, IU/L):** The detected value is 83, which is outside the normal range of 10 - 45.
7. **Aspartate aminotransferase (AST, IU/L):** The detected value is 36, which is outside the normal range of 10 - 60.
8. **Gamma-glutamyltransferase (GGT, IU/L):** The detected value is 3,0, which is outside the normal